<a href="https://colab.research.google.com/github/danielad-k/SPRB/blob/master/Capstone%201/1_3_Models_Multiple_Variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import glob
import pandas as pd
import csv
from io import StringIO
import io
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
import nltk
import sys
from scipy.stats import t
from scipy import stats
import statsmodels.api as sa
import statsmodels.formula.api as sfa
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingClassifier
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.svm import SVC
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True



```
# This is formatted as code
```

## 1.Read Data, Data Cleaning


Data Cleaning

In [0]:
kick_data_cleaned = pd.read_csv('kick_data_cleaned.csv',index_col = None)

In [0]:
kick_data_cleaned = kick_data_cleaned.query("state == 'failed' or state == 'successful'") #only focus on successful and failed projects

In [0]:
kick_data_cleaned["launched_deadline_days_diff"] = kick_data_cleaned["launched_deadline_days_diff"] .apply(lambda x: x + 1)

In [0]:
kick_data_cleaned['state_num'] = kick_data_cleaned.state.map({'successful': 1, 'failed': 0 }) #convert to one and zero

In [0]:
kick_data_cleaned = kick_data_cleaned[~kick_data_cleaned.name.isnull()] #remove nulls - they are around 3-4

In [9]:
kick_data_cleaned.head()

,Unnamed: 0,ID,category,main_category,goal,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,launched_month,deadline_month,launched_deadline_days_diff,pledged_goal_diff,goal_outcome,name,launch_deadline_diff,currency,deadline,launched,state_num
0,0,175876.0,Poetry,Publishing,724.0,0.0,failed,0.0,GB,0.0,0.0,11173.0,7.0,9.0,59.0,82468.0,negative,The Songs of Adelaide & Abullah,58 days 11:48:00.000000000,GBP,2015-10-09 00:00:00,2015-08-11 12:12:00,0
1,1,175877.0,Narrative Film,Film & Video,5991.0,8764.0,failed,15.0,US,7354.0,38017.0,38849.0,8.0,10.0,60.0,29783.0,negative,Greeting From Earth: ZGAC Arts Capsule For ET,59 days 19:17:00.000000000,USD,2017-11-01 00:00:00,2017-09-02 04:43:00,0
2,2,175878.0,Narrative Film,Film & Video,6512.0,1195.0,failed,3.0,US,12519.0,11363.0,41904.0,0.0,1.0,45.0,20202.0,negative,Where is Hank?,44 days 23:40:00.000000000,USD,2013-02-26 00:00:00,2013-01-12 00:20:00,0
3,3,175879.0,Music,Music,2552.0,1.0,failed,1.0,US,52.0,54.0,21197.0,2.0,3.0,30.0,66441.0,negative,ToshiCapital Rekordz Needs Help to Complete Album,29 days 20:36:00.000000000,USD,2012-04-16 00:00:00,2012-03-17 03:24:00,0
5,5,175881.0,Restaurants,Food,6658.0,51792.0,successful,224.0,US,88037.0,96324.0,42659.0,1.0,3.0,35.0,124839.0,surplus,Monarch Espresso Bar,34 days 10:22:00.000000000,USD,2016-04-01 00:00:00,2016-02-26 13:38:00,1


In [0]:
kick_data_cleaned['goal_per_day'] = kick_data_cleaned.usd_goal_real/kick_data_cleaned.launched_deadline_days_diff

In [11]:
kick_data_cleaned[['state']].groupby(['state']).size()/len(kick_data_cleaned) 
#there isn't a huge data imbalence between the columns.

state
failed        0.593465
successful    0.406535
dtype: float64

###1.1 POS Preprocesing

In [0]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent) #tokenize each word
    sent = nltk.pos_tag(sent) # get pos tag
    return sent

In [0]:
kick_data_cleaned['tag'] = kick_data_cleaned['name'].apply(lambda x: preprocess(x))

In [0]:
def prep2(x):
  import re
  tup = [j[1] for j in x] #extract post tag
  tupx = sorted(set(tup)) # get unique items
  tup2 = ','.join(tupx) #join
  tup1 = re.sub(":|\.|\$|``|\)|\(|''|#",'',tup2) #remove :, $, #
  tup3 = re.sub('^,{1,50}|,$','',tup1) #remove space

  return tup3

In [0]:
kick_data_cleaned['tag2'] = kick_data_cleaned['tag'].apply(lambda x: prep2(x))

In [16]:
kick_data_cleaned.shape

(127347, 26)

In [0]:
dummies = kick_data_cleaned.tag2.str.get_dummies(',') #get dummies

In [0]:
dummies = pd.DataFrame(dummies) # convert to dataframe

In [0]:
kick_data_cleaned =pd.concat([kick_data_cleaned,dummies],axis = 1)

###1.2 Select Columns and Traing/Test Split

In [0]:
X = kick_data_cleaned[kick_data_cleaned.columns.difference(["goal","ID","state_num","pledged","backers","usd_pledged_real","pledged_goal_diff",
                              "goal_outcome","launch_deadline_diff","currency","deadline","launched","state","tag","tag2","Unnamed: 0","usd pledged"])]

In [0]:
X = X.fillna(0)

In [0]:
y = kick_data_cleaned.state_num 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #test_train model

## 2.Pipeline: Basic Model


Test which model tends to have the best performance.


In [0]:
from sklearn.base import BaseEstimator

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

In [0]:
from sklearn.base import TransformerMixin
from nltk.corpus import stopwords

class LemStem(TransformerMixin):
    def lem(self):
        return WordNetLemmatizer().lemmatize(self, pos='v')
            
    
    def stem(self):
        return PorterStemmer().stem(self)
    
    def __call__(self,x):
        return [LemStem.stem(t.lower()) for t in x]
    
    def fit(self):
        pass
    
    def transform(self):
        pass
        
    
 
        
        

In [0]:
#preprocessing - make into one hot encoding
categorical_features = ['main_category','category', 'country']
categorical_transformer = Pipeline(steps=[
   ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

In [0]:
numeric_features = ['usd_goal_real','goal_per_day']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [0]:
#vectorize categorical variables
vect_features = 'name'
vect_transformer = Pipeline(steps=[
    ('vect', CountVectorizer(strip_accents = 'ascii',
                             ngram_range = (1,2)))])

In [0]:
#Use column transformer to join them
preprocessor = ColumnTransformer(
    transformers=[
        ('vect', vect_transformer, vect_features),
        ('cat', categorical_transformer, categorical_features),
         ('num', numeric_transformer,numeric_features)])

In [0]:
#create pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('clf', DummyEstimator())])

In [0]:
search_space = [ #input 8 classifiers 
                    
                 {
                  'clf': [xgb.XGBClassifier()]},
                {'clf': [linear_model.SGDClassifier(max_iter=1000, tol=1e-3)]},
                {
                 'clf': [RandomForestClassifier(max_depth=5,random_state=0)]
                } ,
                {
                 'clf': [SVC(max_iter = 1000)]},
    

    
                {
                  'clf':[AdaBoostClassifier()]},
            
                {
                    'clf': [LogisticRegression()]},

                                

]

In [69]:
#based on grid search then make 
grid = GridSearchCV(pipe, cv=5, param_grid=search_space,error_score = 0,verbose = 10, n_jobs = -1)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.7min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 16.0min finished


GridSearchCV(cv=5, error_score=0,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('vect',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('vect',
                                                                                          CountVectorizer(analyzer='word',
                                                                                                          binary=False,
                                     

In [70]:
print("Best: %f using %s" % (grid.best_score_, 
    grid.best_params_))

Best: 0.693831 using {'clf': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)}


In [71]:
pd.DataFrame(grid.cv_results_)[['param_clf','mean_test_score','rank_test_score']].sort_values(by = 'rank_test_score')

,param_clf,mean_test_score,rank_test_score
1,"SGDClassifier(alpha=0.0001, average=False, cla...",0.693831,1
5,"LogisticRegression(C=1.0, class_weight=None, d...",0.683218,2
0,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.671697,3
4,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",0.670061,4
2,"RandomForestClassifier(bootstrap=True, ccp_alp...",0.595997,5
3,"SVC(C=1.0, break_ties=False, cache_size=200, c...",0.488846,6


In [0]:
pipe.set_params(**grid.best_params_)
pipe.fit(X_train,y_train)
y_preds = pipe.predict(X_test)

In [73]:
print('Accuracy score on test set %.3f' % accuracy_score(y_test, y_preds))

Accuracy score on test set 0.694


### 2.1 Feature Influence

In [0]:
#get the column names for the vectorizer
prep_doc = []
for p in preprocessor.transformers_[0][1]['vect'].get_feature_names():
    prep_doc.append(p)

In [0]:
#get column names for onehot encoding and save into a list
prep_doc_oh = preprocessor.transformers_[1][1]['onehot'].get_feature_names().tolist()

In [0]:
prep_num = preprocessor.transformers_[2][2]

In [0]:
#combine them in one list
prep = prep_doc + prep_doc_oh + prep_num

In [78]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [79]:

import eli5
eli5.show_weights(pipe[1],top = 50, feature_names = prep)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Weight?,Feature
+1.381,x1_Tabletop Games
+1.134,residency
+1.046,pens
+1.017,x0_Theater
+1.014,x1_Shorts
+1.004,watches
+0.963,visual novel
+0.916,x1_Classical Music
+0.906,zine
+0.874,titanium


###2.2 Precision and Recall 

In [80]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_preds, labels=range(2)))
print(classification_report(y_test, y_preds, target_names=['failed','successful']))

[[33171  6405]
 [13874 12885]]
              precision    recall  f1-score   support

      failed       0.71      0.84      0.77     39576
  successful       0.67      0.48      0.56     26759

    accuracy                           0.69     66335
   macro avg       0.69      0.66      0.66     66335
weighted avg       0.69      0.69      0.68     66335



Low recall for successful - more likely to predict that the result is success compared to actual success. 


## 3.HyperParametrization

Since SGC classifier has performed the best, we are looking at whether we can fine tune it.

In [0]:
search_space_sgd= [{ 'clf': [linear_model.SGDClassifier()],
                 'clf__l1_ratio':[0.1,0.5,0.7],
                 'clf__penalty':['l1','l2','elasticnet'],
                 'clf__alpha': [0.0001,0.01,0.05,0.5,1]}
                                ]

In [82]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'clf', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__vect', 'preprocessor__cat', 'preprocessor__num', 'preprocessor__vect__memory', 'preprocessor__vect__steps', 'preprocessor__vect__verbose', 'preprocessor__vect__vect', 'preprocessor__vect__vect__analyzer', 'preprocessor__vect__vect__binary', 'preprocessor__vect__vect__decode_error', 'preprocessor__vect__vect__dtype', 'preprocessor__vect__vect__encoding', 'preprocessor__vect__vect__input', 'preprocessor__vect__vect__lowercase', 'preprocessor__vect__vect__max_df', 'preprocessor__vect__vect__max_features', 'preprocessor__vect__vect__min_df', 'preprocessor__vect__vect__ngram_range', 'preprocessor__vect__vect__preprocessor', 'preprocessor__vect__vect__stop_words', 'preprocessor__vect__vect__strip_accents', 'preprocessor__vect__vect__token_p

In [83]:
grid_sgd = GridSearchCV(pipe, cv=5, param_grid=search_space_sgd,error_score = 0,verbose = 10,n_jobs = -1)
grid_sgd.fit(X_train,y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 15

GridSearchCV(cv=5, error_score=0,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('vect',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('vect',
                                                                                          CountVectorizer(analyzer='word',
                                                                                                          binary=False,
                                     

In [84]:
print("Best: %f using %s" % (grid_sgd.best_score_, 
    grid_sgd.best_params_))

Best: 0.695071 using {'clf': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.1, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False), 'clf__alpha': 0.0001, 'clf__l1_ratio': 0.1, 'clf__penalty': 'l2'}


In [0]:
pipe.set_params(**grid_sgd.best_params_)
pipe.fit(X_train,y_train)
y_preds = pipe.predict(X_test)

In [86]:
print('Accuracy score on test set %.3f' % accuracy_score(y_test, y_preds))

Accuracy score on test set 0.695


The hyperparametrizied model has same accuracy as the baseline model therefore we stick with baseline model.

## 4.Stacking Model

In [0]:
 estimators = [
               ('xgb', xgb.XGBClassifier()),
                ('ada', AdaBoostClassifier())]
               


In [0]:
stack = StackingClassifier(estimators=estimators, final_estimator=linear_model.SGDClassifier(max_iter=500, tol=1e-3))



In [0]:
pipe_stack = Pipeline(steps=[('preprocessor', preprocessor),
                      ('stack', stack)])

In [0]:
pgrid = [{'preprocessor__vect__vect__ngram_range': ((1, 1),(1, 2))}]

In [35]:
grid_st = GridSearchCV(pipe_stack, cv=3,param_grid = pgrid,error_score = 0, verbose = 10,n_jobs = -1)
grid_st.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 124.9min remaining: 62.5min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 185.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 185.4min finished


GridSearchCV(cv=3, error_score=0,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('vect',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('vect',
                                                                                          CountVectorizer(analyzer='word',
                                                                                                          binary=False,
                                     

In [36]:
print("Best: %f using %s" % (grid_st.best_score_, 
    grid_st.best_params_))

Best: 0.670043 using {'preprocessor__vect__vect__ngram_range': (1, 1)}


In [37]:
pipe_stack.set_params(**grid_st.best_params_)
pipe_stack.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('vect',
                                                  Pipeline(memory=None,
                                                           steps=[('vect',
                                                                   CountVectorizer(analyzer='word',
                                                                                   binary=False,
                                                                                   decode_error='strict',
                                                                                   dtype=<class 'numpy.int64'>,
                                                                                   encoding='utf-8',
                                        

In [0]:
y_preds = pipe_stack.predict(X_test)

In [39]:
print('Accuracy score on test set %.3f' % accuracy_score(y_test, y_preds))

Accuracy score on test set 0.675


The stacking model is almost the same as the baseline model.